In [1]:
import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
data = pd.read_csv(r'/home/supriya/pathcontext/vectorization_work/CDS_vectorization_v1.csv',dtype={'Error_Label':object} )
print(data)
#data.head()

          ID       v1       v2       v3       v4       v5       v6       v7  \
0      10001  0.41463  1.18500  1.12590 -0.63784 -1.60940 -0.19869 -0.23010   
1      10002 -1.68950 -0.63024 -0.23417 -2.21810 -0.36157 -0.52164 -0.31883   
2      10003 -1.71670  2.40120  0.44570  1.14400 -0.53560 -0.69040  0.39990   
3      10004 -0.07260  0.31790 -1.38690  1.83000 -1.08930 -1.15900  0.05750   
4      10005  0.32088 -1.08450 -1.15780 -0.63845  1.03060  0.38707  0.95113   
...      ...      ...      ...      ...      ...      ...      ...      ...   
62178  96207 -1.45150 -0.95600 -0.89920 -0.80340 -0.83730  2.07520 -2.71140   
62179  96208  1.27570 -1.32360  2.00620 -1.61870 -0.72743  0.58177 -2.55890   
62180  96209  0.35809 -2.22570  0.22366 -2.89240  0.43210 -1.64220 -0.12659   
62181  96210  0.40804 -2.27950 -0.83962 -1.03440  1.04950  1.32130  1.27670   
62182  96211 -0.74982  0.35505 -0.47464 -2.38330 -1.45550 -0.62356 -1.79640   

            v8       v9  ...      v491     v492    

In [2]:
# split labels and return new dataframe
#Create independent and Dependent Features
#data=split_label(data)
def split_label_dict(df):
    # Change 'label' to whatever naming used in the original DataFrame.
    temp_df = df['Error_Label'].str.split("", n = -1, expand = True)
    for i in range(1, 19):
        df[f'l{1}'] = temp_df[1]
        df[f'l{1}'] = pd.to_numeric(df[f'l{1}'])
    return df

newdata=split_label_dict(data)
#Create independent and Dependent Features
columns = newdata.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["l1"]]
# Store the variable we are predicting 
target = "l1"
# Define a random state 
state = np.random.RandomState(69)
X = newdata[columns]
Y = newdata[target]
# Print the shapes of X & Y
print(X.shape)
print(Y)

(62183, 501)
0        0
1        0
2        0
3        0
4        0
        ..
62178    0
62179    0
62180    0
62181    0
62182    0
Name: l1, Length: 62183, dtype: int64


In [3]:

from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=69,stratify=Y)


In [4]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [8]:

# Fitting Kernel SVM to the Training set
from sklearn.svm import SVC
classifier = SVC()
classifier.fit(X_train, y_train)

SVC()

In [ ]:
# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)